In [1]:
import numpy as np
import torch
import torch.nn as nn

Only understands tensor.....

In [2]:
a = np.array([1,2,3,4])

In [3]:
b = [1,2,3,4]

In [4]:
a_t = torch.tensor(a,dtype=torch.long)

In [5]:
a_t

tensor([1, 2, 3, 4])

In [6]:
b_t = torch.tensor(b,dtype=torch.long)

In [7]:
b_t

tensor([1, 2, 3, 4])

In [8]:
b_t.numpy()

array([1, 2, 3, 4], dtype=int64)

Reshaping tensors...

In [13]:
a = torch.randn(3,4)
a.size()

torch.Size([3, 4])

In [14]:
a.view(-1,1).size()

torch.Size([12, 1])

In [15]:
b = a.view(1,-1)

In [16]:
b.shape

torch.Size([1, 12])

In [25]:
s = b.squeeze(0)
s.shape
# b.unsqueeze(0)

torch.Size([12])

In [29]:
n = a.numpy()
m = torch.from_numpy(n)
m

tensor([[ 0.0763, -0.9103,  0.7906, -0.0368],
        [-0.3423, -0.5396,  0.3462,  0.8418],
        [-0.5109, -0.5894,  0.3872,  0.2229]])

Variables....

In [30]:
from torch.autograd import Variable

In [31]:
var = Variable(torch.rand(3,4),requires_grad=True)

In [32]:
var

tensor([[0.5689, 0.1197, 0.8649, 0.0994],
        [0.3705, 0.3497, 0.7421, 0.7109],
        [0.9873, 0.8100, 0.0532, 0.2403]], requires_grad=True)

In [33]:
a = torch.ones((3,4),requires_grad=True)

In [34]:
a

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [36]:
op = torch.sum(a+var)
op

tensor(17.9170, grad_fn=<SumBackward0>)

In [41]:
op.backward()

In [42]:
op.grad_fn

In [43]:
var.grad

tensor([[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]])

In [44]:
b = var.detach()

In [45]:
b.requires_grad

False

In [46]:
var.requires_grad

True

Add paramters to a model

In [43]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model,self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.W1 = nn.Parameter(torch.randn(self.input_size, self.hidden_size, requires_grad=True))
        self.b1 = nn.Parameter(torch.randn(1, self.hidden_size, requires_grad=True))
        #self.relu = nn.ReLU()
        self.W2 = nn.Parameter(torch.randn(self.hidden_size, self.hidden_size, requires_grad=True))
        self.b2 = nn.Parameter(torch.randn(1, self.hidden_size, requires_grad=True))
        #self.W3 = nn.Parameter(torch.randn(self.hidden_size, self.output_size, requires_grad=True))
        #self.b3 = nn.Parameter(torch.randn(1, self.output_size, requires_grad=True))
        #self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x @ self.W1 + self.b1
        x = self.relu(x)
        # second hidden layer
        x = x @ self.W2 + self.b2
        x = self.relu(x)
        # output layer
        x = x @ self.W3 + self.b3 # => logits
        x = self.softmax(x)

        return x

In [44]:
model = Model(25,50,5)

In [47]:
inp = torch.randn(4,25)

In [48]:
model(inp)

tensor([[2.3556e-42, 1.5756e-34, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00, 4.3740e-18, 3.8724e-17],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.6242e-12, 1.0000e+00]],
       grad_fn=<SoftmaxBackward>)

Dealing with batches...

In [47]:
x_seq = torch.tensor([[1, 2, 4, 4], [2, 3, 4 ,0], [2, 3, 0, 0]],dtype=torch.long)
x_len = torch.tensor([4,3,2])

In [48]:
embedding = nn.Embedding(5, 10, padding_idx=0)

In [49]:
x_seq.shape

torch.Size([3, 4])

In [50]:
from torch import nn
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

In [53]:
inp = embedding(x_seq)
inp

tensor([[[ 0.9177, -1.4231, -0.8122,  1.7667, -1.0171, -1.3062, -1.0722,
           0.1679, -0.2207, -0.0051],
         [ 3.0232, -0.0281, -0.0244, -1.1532,  0.7851, -0.5970,  1.0270,
          -0.3429, -0.2520, -1.2006],
         [ 0.4329, -0.1075,  0.6592, -0.5811,  1.4113,  1.6327,  0.3199,
           1.3880, -0.7371,  0.1626],
         [ 0.4329, -0.1075,  0.6592, -0.5811,  1.4113,  1.6327,  0.3199,
           1.3880, -0.7371,  0.1626]],

        [[ 3.0232, -0.0281, -0.0244, -1.1532,  0.7851, -0.5970,  1.0270,
          -0.3429, -0.2520, -1.2006],
         [-0.4264, -1.4731,  0.3802,  1.4937, -0.7627,  0.8810,  0.4608,
          -0.1497, -0.3878,  1.2862],
         [ 0.4329, -0.1075,  0.6592, -0.5811,  1.4113,  1.6327,  0.3199,
           1.3880, -0.7371,  0.1626],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000]],

        [[ 3.0232, -0.0281, -0.0244, -1.1532,  0.7851, -0.5970,  1.0270,
          -0.3429, -0.2520, -1.2006

In [52]:
inp.shape

torch.Size([3, 4, 10])

In [121]:
inp

tensor([[[ 0.8704, -0.5220,  0.2783,  1.2949,  2.1186, -1.2534,  0.2117,
           1.3384, -1.4492,  0.4275],
         [ 0.1250,  1.2673, -2.3303,  0.0739, -0.1839, -1.3386,  0.4062,
           0.0080,  0.8248, -0.6236],
         [ 1.8905, -0.4285,  1.0940,  0.4310, -0.2811, -1.9780,  2.0421,
          -0.0735,  0.3303,  0.1926],
         [ 1.8905, -0.4285,  1.0940,  0.4310, -0.2811, -1.9780,  2.0421,
          -0.0735,  0.3303,  0.1926]],

        [[ 0.1250,  1.2673, -2.3303,  0.0739, -0.1839, -1.3386,  0.4062,
           0.0080,  0.8248, -0.6236],
         [-2.2452,  1.0721, -1.1496, -1.3051,  0.3180, -0.1924, -1.0166,
          -0.7855,  0.1190,  0.0629],
         [ 1.8905, -0.4285,  1.0940,  0.4310, -0.2811, -1.9780,  2.0421,
          -0.0735,  0.3303,  0.1926],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000]],

        [[ 0.1250,  1.2673, -2.3303,  0.0739, -0.1839, -1.3386,  0.4062,
           0.0080,  0.8248, -0.6236

In [54]:
x_padded = pack_padded_sequence(inp, x_len, batch_first=True)

In [55]:
gru = nn.GRU(10, 10, batch_first=True)

In [56]:
hidden = torch.zeros(1, 3, 10)

In [57]:
out, hidden = gru(x_padded, hidden)

In [58]:
output,len_ = pad_packed_sequence(out, batch_first=True)

In [59]:
output.shape

torch.Size([3, 4, 10])

In [60]:
output

tensor([[[ 0.2696,  0.2997,  0.0895,  0.0050,  0.0496, -0.3096,  0.3624,
          -0.4576, -0.2141,  0.3659],
         [ 0.2773, -0.0688,  0.1000, -0.1101,  0.2123, -0.0726,  0.5606,
           0.2856, -0.5113, -0.2201],
         [ 0.4325, -0.3911,  0.3640, -0.1265,  0.5877,  0.0876,  0.6446,
           0.3820, -0.2831, -0.3755],
         [ 0.4625, -0.5619,  0.5197, -0.1188,  0.7566,  0.1639,  0.6919,
           0.4501, -0.1263, -0.4647]],

        [[ 0.0839, -0.2508,  0.0065, -0.0645,  0.1801,  0.1025,  0.2999,
           0.4622, -0.3415, -0.3251],
         [ 0.3483,  0.1941,  0.2423,  0.0744,  0.3573, -0.0199,  0.0944,
          -0.1792, -0.1099, -0.0070],
         [ 0.4403, -0.2516,  0.4334, -0.0169,  0.6555,  0.0329,  0.3395,
           0.1182, -0.1124, -0.2392],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000]],

        [[ 0.0839, -0.2508,  0.0065, -0.0645,  0.1801,  0.1025,  0.2999,
           0.4622, -0.3415, -0.3251

In [61]:
len_

tensor([4, 3, 2])